# INF8775 – Analyse et conception d’algorithmes
# TP3 – Hiver 2024

Nguyen, Huy Viet, 2136374

Some, Freddy, 1930443

Note finale :

 <u>**Date limite de remise :**</u>  17 avril 23h59 (Groupe B1), 9 avril 23h59 (Groupe B2)

# Instructions

## Rédaction et remise du rapport

- Ce notebook constitue à la fois le sujet du TP, votre code et votre rapport. Il contient déjà du code pour faciliter vos mesures et l'affichage de vos résultats, ainsi qu'un squelette pour votre rapport.

- Complétez directement le notebook, vous êtes libres de créer de nouvelles cellules de code ou de texte. 

- Remettez le fichier du notebook sur Moodle avec le nom `NOM1_MATRICULE1_NOM2_MATRICULE2.ipynb`

- Vous pouvez inclure du code trouvé sur Internet, mais vous devez en mentionner la source, sous peine d'être sanctionnés pour plagiat.

## Mise en situation

Le dernier travail pratique se fera dans le cadre du concours du meilleur algorithme pour la session d'hiver 2024. Vous devez concevoir et implanter un algorithme de votre cru pour résoudre un problème combinatoire. Le classement des équipes déterminera votre note pour la qualité de l'algorithme. Votre algorithme sera exécuté sur 3 exemplaires de notre choix, chacun pendant 1 minute.<br>
Le rapport pour ce dernier travail pratique est assez succinct. Nous vous encourageons à terminer ce travail assez tôt afin de ne pas compromettre votre préparation à vos examens finaux.

## Description du problème

Le problème à résoudre est le suivant : Vous disposez d'un ensemble de blocs de dimensions variées en  hauteur, largeur et  profondeur décris par un triplet $(l, p, h)$ (largeur, profondeur, hauteur). Afin de garantir la stabilité de votre tour, vous devez vous assurer que le bloc que vous ajoutez sur votre tour repose entièrement sur le précédent.

Plus formellement, les contraintes suivantes doivent être respectées :

$$l_{nouveauBloc} \leq l_{blocReceveur}  \text{ et  } p_{nouveauBloc} \leq p_{blocReceveur}$$

On vous demande de construire plusieurs tours en utilisant tous les blocs d'une instance. En plus des contraintes d'équilibre, les tours ne doivent pas dépasser une hauteur maximum H. L'objectif est de minimiser le nombre de tours construites tout en respectant les contraintes décrites précédemment. Il est interdit de faire une rotation des blocs. Vous pouvez vérifier si votre solution est valide avec la fonction suivante :

In [1]:
def check_solution(sample_blocks, H, solution):
    maxHeightOK, blockSmallerOK, allBlocksUsed, onlyGivenBlocks = True, True, True, True
    print(f"Nombre de tours utilisées : {len(solution)}")

    for tower in solution:
        if sum([b[2] for b in tower]) > H:
            maxHeightOK = False
            break
    for t,tower in enumerate(solution):
        for i in range(len(tower) - 1):
            new_block = tower[i + 1]
            receiving_block = tower[i]
            if new_block[0] > receiving_block[0] or new_block[1] > receiving_block[1]:
                blockSmallerOK = False
                break
    print(f"Hauteur maximale non dépassée : {maxHeightOK}\nPas de blocs plus gros reposant sur un plus petit : {blockSmallerOK}")
    
    solution_blocks = [b for t in solution for b in t]
    block_counts = {}
    for b in sample_blocks:
        if b not in block_counts.keys():
            block_counts[b] = 0
        block_counts[b] += 1
    
    for b in solution_blocks:
        if b not in block_counts.keys():
            onlyGivenBlocks = False
            break
        block_counts[b] -= 1
        
    for count in block_counts.values():
        if count > 0:
            allBlocksUsed = False
        if count < 0:
            onlyGivenBlocks = False
        if not (allBlocksUsed and onlyGivenBlocks):
            break
    print(f"Tous les blocs sont utilisés : {allBlocksUsed}\nUniquement les blocs du sample sont utilisés : {onlyGivenBlocks}")

## Exemple

Pour l'exemplaire de 4 blocs suivant : `[(1, 5, 8),(9, 6, 2), (8, 5, 4),(6, 2, 1)]`

On évalue la solution de 2 tours (remarquez la liste de liste) :

```
[
    [(1, 5, 8)],
    [(9, 6, 2),(8, 5, 4), (6, 2, 1)]
]
```

In [2]:
s = [(1, 5, 8), (9, 6, 2), (8, 5, 4), (6, 2, 1)]
check_solution(s, 9, [[(1,5,8)], [(9, 6, 2),(8,5,4),(6,2,1)]])

Nombre de tours utilisées : 2
Hauteur maximale non dépassée : True
Pas de blocs plus gros reposant sur un plus petit : True
Tous les blocs sont utilisés : True
Uniquement les blocs du sample sont utilisés : True



## Jeu de données

La fonction `generate_sample` ci-dessous permet de générer une liste de N blocs de dimensions variables.



In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
from random import randint

In [4]:
def generate_sample(size, max_width=100, max_depth=100, max_height=20):
    return [
    (randint(1, max_width), randint(1, max_depth), randint(1, max_height))
    for _ in range(size)
    ]

def writeFile(sample, H, filePath):
    f = open(filePath, "w+")
    f.write(f"{H}\n")
    for w, p, h in sample:
        f.write(f"{w} {p} {h}\n")
    f.close()

def readFile(filePath):
    """
    Charger une instance depuis un fichier
    """
    f = open(filePath)
    lines = list(f)
    H = int(lines[0])
    blocks = [
        tuple(map(int, x.split(' ')))
        for x in lines[1:]
    ]
    f.close()
    return H, blocks

Utilisez la cellule ci-dessous pour vous créer des exemplaires de tailles variables, que vous pourrez réaccéder plus tard à l'aide de readFile()

In [5]:
writeFile(generate_sample(size=10, max_width=100, max_depth=100, max_height=10), 15, 'instance1.txt')

## Votre algorithme 

### Présentation

Présentez votre algorithme sous forme de pseudo-code et incluant une analyse de complexité théorique des principales fonctions.

*Insérer votre réponse ici*

### Implémentation

Implémentez votre algorithme ci-dessous. Veillez à respecter le nom de la fonction, ses arguments, ainsi que le format de sortie.

In [37]:
import time
import itertools
import random
import math
from typing import Callable

In [129]:
def greedy_solve(blocks: tuple[int, int, int], H: int, timeout: Callable[[], bool]):
    towers = []
    heights = []
    
    def add_block(block):     
        if timeout(): 
            return
    
        for i, tower in enumerate(towers):
            if heights[i] + block[2] <= H and tower[-1][1] >= block[1]:
                tower.append(block)
                heights[i] += block[2]
        
                return
        
        towers.append([block])
        heights.append(block[2])
        
        return

    blocks.sort(key=lambda x : (x[0], x[1]), reverse=True)     
    for block in blocks:
        add_block(block)
        
    return towers, heights

In [192]:
def local_search(towers, heights, H, timeout, iteration_factor=None):
    if iteration_factor is None:
        iteration_factor = 2000

    search_iterations = iteration_factor * len(towers)

    def random_swap_blocks(i, j):
        if timeout(): return
        
        tower = towers[i]
        other_tower = towers[j]

        index = random.randint(0, len(tower)-1)        
        block_to_swap = tower[index]
        
        if heights[j] + block_to_swap[2] > H:
            return

        if block_to_swap[0] <= other_tower[-1][0] and block_to_swap[1] <= other_tower[-1][1]:
            other_tower.append(tower.pop(index))
            heights[j] += block_to_swap[2]
            heights[i] -= block_to_swap[2]

        else:
            for k, block in enumerate(other_tower[:-1]):
                if timeout(): return
    
                higher_block = other_tower[k+1]
    
                if higher_block[0] <= block_to_swap[0] <= block[0] and higher_block[1] <= block_to_swap[1] <= block[1]:
                    other_tower.insert(k+1, tower.pop(index))
                    heights[j] += block_to_swap[2]
                    heights[i] -= block_to_swap[2]

                    break

        if not tower:
            towers.pop(i)
            heights.pop(i)
        
        return
    
    for i in range(search_iterations):
        if timeout(): return

        i, j = random.choices(range(0, len(heights)), [H - height for height in heights], k=2)
        if i == j:
            j = (j + 1) % len(heights)
            
        random_swap_blocks(i, j)

    return

In [ ]:
def elimination_search(towers, heights, H, timeout):
    
    return

In [196]:
def solve(blocks, H, maxTime=60):
    """
    Retourne la liste des tours construites.
    Une tour est une liste de bloc, où le bloc tour[i+1] est posé sur le bloc tour[i]
    Un bloc est un tuple (largeur, profondeur, hauteur)
    """
    start = time.time()
    towers = []
    heights = []
    
    timeout = lambda: time.time() - start > 60 - 0.25

    towers, heights = greedy_solve(blocks, H, timeout)

    if timeout(): return towers

    # local_search(towers, heights, H, timeout) Local search inefficace

    return towers

In [198]:
H, blocks = readFile('instance3.txt')
print("maxHeight :",H, "\nNumber of blocks :",len(blocks),'\n')

start = time.time()
solution = solve(blocks, H)
print("Temps:", time.time() - start)
#print(solution)
check_solution(blocks, H, solution)

maxHeight : 800 
Number of blocks : 20000 

Temps: 0.27446627616882324
Nombre de tours utilisées : 268
Hauteur maximale non dépassée : True
Pas de blocs plus gros reposant sur un plus petit : True
Tous les blocs sont utilisés : True
Uniquement les blocs du sample sont utilisés : True


### Justification des choix de conception

La conception de votre algorithme sera jugé avec les critères suivants:

- Lien avec le contenu du cours
- Originalité
- Initiatives

Une approche par force brute est peu pratique pour ce problème puisque le temps de calcul requis sera probablement trop grand. En effet, si on suppose qu'on regarde tous les combinaisons de tours possibles, on aura une complexité de l'ordre de l'exponentielle.



### Votre algorithme est-il certain de trouver une solution optimale ? 

*Insérer votre réponse ici*

## Barême complémentaire

### Qualité de l'algorithme (i.e des solutions) : /4
<p style='text-align: justify;'> 
Vos algorithmes seront testés pendant 1 minute sur 3 exemplaires cachés. D'abord, vous gagnerez 1pt si votre algorithme est capable de battre notre algorithme baseline sur chacun des 3 exemplaires. Les 3 points restants seront distribués en fonction de votre classement par rapport aux autres équipes. Etre dans le premier quartile vous donne 1pt par exemplaire, le 2e quartile 0.75pts etc... et 0.25pts pour le dernier quartile. Ainsi si vous battez la baseline, vous êtes surs d'avoir au minimum 1.75/4 pts pour cette partie. <br>
L'évaluation prend également en compte le temps d'execution, avec une forte bonification pour les algorithmes inférieurs à 5 secondes, et converge vers pas de bonification pour les algorithmes durant 1 minutes : 
</p>

$$ Score = - \min(1.5, 1+\frac{5}{2t}) \times N_{towers} $$

Si votre algorithme ne renvoie pas de solution dans le temps imparti ou que la solution n'est pas valide, alors vous n'obtiendrez pas de point pour cet exemplaire.

Qualité du code : /1

Présentation générale (concision, qualité du français): /1